In [6]:
# Импорт библиотек
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import requests
from dotenv import load_dotenv, find_dotenv
import os

In [7]:
# Загрузка переменных окружения
dotenv_path = find_dotenv()
if dotenv_path:
    load_dotenv(dotenv_path)
else:
    print("Файл .env не найден")

In [8]:
# Проверка наличия переменной окружения API_URL
api_url = os.getenv("API_URL")
if not api_url:
    raise ValueError("Переменная окружения API_URL не найдена. Проверьте файл .env")


In [9]:
# Функции для запроса данных по API
def fetch_visits(api_url, start_date, end_date):
    response = requests.get(f"{api_url}/visits", params={'begin': start_date, 'end': end_date})
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

def fetch_registrations(api_url, start_date, end_date):
    response = requests.get(f"{api_url}/registrations", params={'begin': start_date, 'end': end_date})
    if response.status_code == 200:
        return response.json()
    else:
        response.raise_for_status()

In [10]:
# Запрос данных
start_date = "2023-03-01"
end_date = "2023-09-01"

visits_data = fetch_visits(api_url, start_date, end_date)
registrations_data = fetch_registrations(api_url, start_date, end_date)


In [11]:
# Преобразование данных в DataFrame
visits_df = pd.DataFrame(visits_data)
registrations_df = pd.DataFrame(registrations_data)

In [12]:
# Удаление визитов ботов
visits_df = visits_df[~visits_df['user_agent'].str.contains('bot')]

# Преобразование колонок datetime в тип datetime
visits_df['datetime'] = pd.to_datetime(visits_df['datetime'])
registrations_df['datetime'] = pd.to_datetime(registrations_df['datetime'])

# Добавление колонки date_group, которая содержит только дату (без времени)
visits_df['date_group'] = visits_df['datetime'].dt.date
registrations_df['date_group'] = registrations_df['datetime'].dt.date


In [13]:
# Группировка данных визитов
visits_agg = visits_df.groupby(['date_group', 'platform']).size().reset_index(name='visits')

# Группировка данных регистраций
registrations_agg = registrations_df.groupby(['date_group', 'platform']).size().reset_index(name='registrations')

# Объединение данных визитов и регистраций
merged_df = pd.merge(visits_agg, registrations_agg, on=['date_group', 'platform'], how='left')
merged_df['registrations'] = merged_df['registrations'].fillna(0)  # Заполнение NaN значений в регистрациях нулями

# Расчет конверсии
merged_df['conversion'] = (merged_df['registrations'] / merged_df['visits']) * 100

# Сортировка данных по дате
merged_df = merged_df.sort_values(by='date_group')

# Сохранение данных в формате JSON
merged_df.to_json('conversion.json', orient='records', date_format='iso')


In [14]:
# Загрузка данных о рекламных кампаниях из CSV
ads_df = pd.read_csv('ads.csv')

# Преобразование колонки date в тип datetime и выделение только даты
ads_df['date'] = pd.to_datetime(ads_df['date']).dt.date

# Агрегирование данных о рекламных кампаниях по дате
ads_agg = ads_df.groupby('date').agg({'cost': 'sum', 'utm_campaign': 'first'}).reset_index()

# Переименование колонки date для соответствия с date_group в основном датафрейме
ads_agg = ads_agg.rename(columns={'date': 'date_group'})
ads_agg['utm_campaign'] = ads_agg['utm_campaign'].fillna('none')


In [15]:
# Объединение данных по дате и платформам
final_df = pd.merge(merged_df, ads_agg, on='date_group', how='left')

# Заполнение NaN значений в затратах и названиях кампаний
final_df['cost'] = final_df['cost'].fillna(0)
final_df['utm_campaign'] = final_df['utm_campaign'].fillna('none')

# Сохранение итогового датафрейма в формате JSON
final_df.to_json('ads.json', orient='records', date_format='iso')


In [16]:
# Проверка сохранения файла
with open('ads.json', 'r') as f:
    data = f.read()
    print(data[:1000])  # Вывод первых 1000 символов для проверки


[{"date_group":"2023-03-01T00:00:00.000","platform":"android","visits":75,"registrations":61,"conversion":81.3333333333,"cost":212.0,"utm_campaign":"advanced_algorithms_series"},{"date_group":"2023-03-01T00:00:00.000","platform":"ios","visits":22,"registrations":18,"conversion":81.8181818182,"cost":212.0,"utm_campaign":"advanced_algorithms_series"},{"date_group":"2023-03-01T00:00:00.000","platform":"web","visits":844,"registrations":8,"conversion":0.9478672986,"cost":212.0,"utm_campaign":"advanced_algorithms_series"},{"date_group":"2023-03-02T00:00:00.000","platform":"android","visits":67,"registrations":59,"conversion":88.0597014925,"cost":252.0,"utm_campaign":"advanced_algorithms_series"},{"date_group":"2023-03-02T00:00:00.000","platform":"ios","visits":31,"registrations":24,"conversion":77.4193548387,"cost":252.0,"utm_campaign":"advanced_algorithms_series"},{"date_group":"2023-03-02T00:00:00.000","platform":"web","visits":1128,"registrations":23,"conversion":2.0390070922,"cost":252.